### Tips

I get **0.87287** with only last year data to train my model, not any hyperparameter turn, So you can use all data and turn some parameter to improve your score. It will be fun.(只使用最后一年的数据训练，使用全部默认参数，得分0.87287，可以修改为使用全部数据，并对年度周期、节假日等超参数进行调整获取更好的结果)

### Other EDA kernel abount this competition

- [M5 Forecasting - Accuracy EDA](https://www.kaggle.com/holoong9291/eda-for-m5-en)
- [M5 Forecasting - Uncertainty EDA](https://www.kaggle.com/holoong9291/eda-for-m5-2-en)

Please upvote me, I am really need a bronze, thanks guys, have a nice day.

# Simple Baseline with Prophet(基于facebook Prophet的baseline模型)

Competition data files introduction(竞赛数据文件介绍)：
- calendar.csv - Contains information about the dates on which the products are sold.(提供商品销售的日期信息)
- sales_train_validation.csv - Contains the historical daily unit sales data per product and store \[d_1 - d_1913\](验证数据集，包含商品信息、门店信息、对应每天的销售额)
- sample_submission.csv - The correct format for submissions. Reference the Evaluation tab for more info.(提交文件样本文件)
- sell_prices.csv - Contains information about the price of the products sold per store and date.(门店的商品在某一天的价格信息)
- sales_train_evaluation.csv - Available once month before competition deadline. Will include sales \[d_1 - d_1941\](评价数据集，在竞赛结束前一个月提供，格式与验证数据集一致)

Target(竞赛目标)： predicting item sales at stores in various locations for two 28-day time periods.(对两个28天的时间段内的门店&商品的销售额进行预测)

Evaluation(评价指标)：RMSSE.(加权的均方根标准误差)

If you like it, please upvote me, and I really need a bronze, and please comment if you have some advise or find my error in this kernel. Let's have fun.(如果你喜欢，请upvote，如果你有任何建议或者发现错误，麻烦再评论区指正，我会尽快完善和修复，谢谢)

In [ ]:
import gc
from tqdm import tqdm
from tqdm._tqdm import trange
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.stats import probplot
from fbprophet import Prophet

%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")

## Take a look(浏览数据)

`Prophet` model is a traditional time series analysis model. It doesn't need or can't make use of price data. For date information, it can be passed in by a way similar to super parameter. But we will do this later. For now, we only consider the simplest prediction of Prophet model.(Prophet模型是传统的时序分析模型，它不需要也无法利用价格数据，对于日期信息，可以通过类似超参数的方式传入，但是这一部分我们放到后面来做，暂时只考虑最最简单的Prophet模型预测)

In [ ]:
folder = '/kaggle/input/m5-forecasting-accuracy/'
calendar = pd.read_csv(folder+'calendar.csv')
validation = pd.read_csv(folder+'sales_train_validation.csv')
submission = pd.read_csv(folder+'sample_submission.csv')

submission = submission[submission.id.str.find('validation')!=-1]
validation = validation.merge(submission, on='id', how='left')
validation = validation.drop(['item_id','dept_id','cat_id','store_id','state_id'], axis=1)

valid_cols = ['d_'+str(1913+i) for i in range(1,29)]
validation.columns = validation.columns.tolist()[:-28]+valid_cols
validation.columns

In [ ]:
submission

In [ ]:
validation

In [ ]:
item1 = validation.iloc[0]
item1 = item1.drop('id').T.reset_index().merge(calendar[['d','date']], left_on='index', right_on='d', how='left').drop(['index','d'], axis=1)
item1.columns = ['y', 'ds']
item1.y = item1.y.astype('float')
item1.ds = item1.ds.astype('datetime64')

rcParams['figure.figsize'] = 20, 5
plt.plot(item1.ds, item1.y)

## Test prophet in one row(在一项上使用Prophet进行预测)

In [ ]:
item = validation.iloc[0]
item = item.drop('id').T.reset_index().merge(calendar[['d','date']], left_on='index', right_on='d', how='left').drop(['index','d'], axis=1)
item.columns = ['y', 'ds']
item.y = item.y.astype('float')
item.ds = item.ds.astype('datetime64')
train_item = item.iloc[:-28]
valid_item = item.iloc[-28:]

ph = Prophet()
ph.fit(train_item)
forecast = ph.predict(item[['ds']])
figure = ph.plot(forecast)
figure.show()

## Predict each store&item in valid date(使用Prophet预测所有门店商品)

It takes about 20 hours to run. It needs to run offline. Just remove `break`.(这里实际运行耗时大约20个小时，去掉break即可).

In [ ]:
for i in trange(len(validation)):
    item = validation.iloc[i]
    item_id = item.id
    item = item.drop('id').T.reset_index().merge(calendar[['d','date']], left_on='index', right_on='d', how='left').drop(['index','d'], axis=1)
    item.columns = ['y', 'ds']
    item.y = item.y.astype('float')
    item.ds = item.ds.astype('datetime64')
    train_item = item.iloc[:-28]
    valid_item = item.iloc[-28:]

    ph = Prophet()
    ph.fit(train_item)
    forecast = ph.predict(valid_item[['ds']])
    validation.iloc[i, -28:] = forecast.yhat.tolist()
    break # FIXME

## Build submission file(构建submission文件)

In [ ]:
submission_prophet = validation[['id']+valid_cols]
submission_prophet.columns = ['id']+['F'+str(i) for i in range(1,29)]
submission_prophet_eval = submission_prophet.copy()
submission_prophet_eval.id = submission_prophet_eval.id.apply(lambda _id:_id.replace('_validation','_evaluation'))
submission_prophet = pd.concat([submission_prophet, submission_prophet_eval])
submission_prophet

In [ ]:
submission_prophet.to_csv('submission_prophet.csv', index=False)

## TODO

- Add date information as hyperparameter in Prophet model.(将日期信息以Prophet的参数形式传入)
- Model parameter adjustment.(模型调参)
- Something I haven't thought about yet.(其他还没想到的点)

The end.